In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from category_encoders import CountEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
 
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
import tensorflow.keras.models as M

In [ ]:
train_df = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
train_df_scored = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
#train_df_noscored = pd.read_csv("/kaggle/input/lish-moa/train_targets_nonscored.csv")

test_df = pd.read_csv("/kaggle/input/lish-moa/test_features.csv")
sample_sub = pd.read_csv("/kaggle/input/lish-moa/sample_submission.csv")

In [ ]:
train_df.head(2)

In [ ]:
test_nulls = test_df['cp_type'] == 'ctl_vehicle'

Уберём айдишники и преобразуем категориальные фичи

In [ ]:
train_df = train_df.iloc[:, 1:]
c_e = CountEncoder(cols=['cp_type', 'cp_dose'])
train_df = c_e.fit_transform(train_df)
train_df.head()

In [ ]:
test_df = test_df.iloc[:, 1:]
test_df = c_e.transform(test_df)
test_df.head()

Введём наш лосс

In [ ]:
def multi_log_loss(y_true, y_pred):
    losses = []
    for col in y_true.columns:
        losses.append(log_loss(y_true.loc[:, col], y_pred.loc[:, col]))
    return np.mean(losses)

In [ ]:
def multi_log_loss_bag(y_true, y_pred):
    losses = []
    for col in y_true.columns:
        try:
            losses.append(log_loss(y_true.loc[:, col], y_pred.loc[:, col]))
        except:
            continue
    return np.mean(losses)

In [ ]:
train_df_scored = train_df_scored.iloc[:, 1:]

In [ ]:
N_FEATS = train_df.shape[1]
N_TARGETS = train_df_scored.shape[1]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_df, train_df_scored, test_size=0.33, random_state=42)

In [ ]:
SEED = 1234
EPOCHS = 15
BATCH_SIZE = 128
LR = 0.0005

In [ ]:
def create_model():
    model = tf.keras.Sequential([
    tf.keras.layers.Input(N_FEATS),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.1),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(1024, activation="relu")),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.7),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(1024, activation="relu")),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(N_TARGETS, activation="sigmoid"))
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = LR), loss='binary_crossentropy', metrics=["accuracy"])
    return model

In [ ]:
model = create_model()

model.fit(train_df,
      train_df_scored,
      epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=2
     )

In [ ]:
pred = model.predict(X_test)

y_pred = y_test.copy()
y_pred.iloc[:, :] = pred

multi_log_loss_bag(y_test, y_pred)

In [ ]:
def create_model():
    model = tf.keras.Sequential([
    tf.keras.layers.Input(N_FEATS),
    tf.keras.layers.BatchNormalization(),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(1024, activation="relu")),
    tf.keras.layers.BatchNormalization(),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(1024, activation="relu")),
    tf.keras.layers.BatchNormalization(),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(N_TARGETS, activation="sigmoid"))
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = LR), loss='binary_crossentropy', metrics=["accuracy"])
    return model

In [ ]:
model = create_model()

model.fit(train_df,
      train_df_scored,
      epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=2
     )

In [ ]:
pred = model.predict(X_test)

y_pred = y_test.copy()
y_pred.iloc[:, :] = pred

multi_log_loss_bag(y_test, y_pred)

In [ ]:
def create_model():
    model = tf.keras.Sequential([
    tf.keras.layers.Input(N_FEATS),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(1024, activation="relu")),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(1024, activation="relu")),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(1024, activation="sigmoid")),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.6),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(N_TARGETS, activation="sigmoid"))
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = LR), loss='binary_crossentropy', metrics=["accuracy"])
    return model

In [ ]:
model = create_model()

model.fit(train_df,
      train_df_scored,
      epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=2
     )

In [ ]:
pred = model.predict(X_test)

y_pred = y_test.copy()
y_pred.iloc[:, :] = pred

multi_log_loss_bag(y_test, y_pred)

In [ ]:
def create_model():
    model = tf.keras.Sequential([
    tf.keras.layers.Input(N_FEATS),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(2048, activation="relu")),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(2048, activation="relu")),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(N_TARGETS, activation="sigmoid"))
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = LR), loss='binary_crossentropy', metrics=["accuracy"])
    return model

In [ ]:
model = create_model()

model.fit(train_df,
      train_df_scored,
      epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=2
     )

In [ ]:
pred = model.predict(X_test)

y_pred = y_test.copy()
y_pred.iloc[:, :] = pred

multi_log_loss_bag(y_test, y_pred)

In [ ]:
def create_model():
    model = tf.keras.Sequential([
    tf.keras.layers.Input(N_FEATS),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(2048, activation="sigmoid")),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(2048, activation="sigmoid")),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(N_TARGETS, activation="sigmoid"))
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = LR), loss='binary_crossentropy', metrics=["accuracy"])
    return model

In [ ]:
model = create_model()

model.fit(train_df,
      train_df_scored,
      epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=2
     )

In [ ]:
pred = model.predict(X_test)

y_pred = y_test.copy()
y_pred.iloc[:, :] = pred

multi_log_loss_bag(y_test, y_pred)

Финальное предсказание

In [ ]:
def create_model():
    model = tf.keras.Sequential([
    tf.keras.layers.Input(N_FEATS),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(2048, activation="relu")),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(2048, activation="relu")),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(N_TARGETS, activation="sigmoid"))
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = LR), loss='binary_crossentropy', metrics=["accuracy"])
    return model

In [ ]:
def create_model():
    model = tf.keras.Sequential([
    tf.keras.layers.Input(N_FEATS),
    tf.keras.layers.BatchNormalization(),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(1024, activation="relu")),
    tf.keras.layers.BatchNormalization(),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(1024, activation="relu")),
    tf.keras.layers.BatchNormalization(),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(N_TARGETS, activation="sigmoid"))
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = LR), loss='binary_crossentropy', metrics=["accuracy"])
    return model

In [ ]:
train_df.head(2)

In [ ]:
SEED = 1234
EPOCHS = 15
BATCH_SIZE = 128
LR = 0.0005

In [ ]:
model = create_model()

model.fit(train_df,
      train_df_scored,
      epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=2
     )

Сохраним предсказание

In [ ]:
pred = model.predict(test_df)

In [ ]:
sample_sub.iloc[:, 1:] = 0

In [ ]:
sample_sub.iloc[:, 1:] = pred

Проставим нули всем, у кого cp_type == 'ctl_vehicle', чтобы улучшить решение


In [ ]:
sample_sub.iloc[:, 1:][test_nulls] = 0

In [ ]:
sample_sub

In [ ]:
sample_sub.to_csv('submission.csv', index=False)